In [19]:
two_image_questions = {k:eval_data[k]['Q'] for k in tqdm(list(eval_data.keys())) if len(eval_data[k]["img_posFacts"]) == 2}
# for k in two_image_questions:
#     if not 'both' in two_image_questions[k]:
#         print(k, two_image_questions[k])


100%|██████████| 504/504 [00:00<00:00, 451694.28it/s]


In [25]:
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model, eval
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init

disable_torch_init()
model_path = "liuhaotian/llava-v1.5-7b"
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, None, model_name
)

def llava_eval_on_webqa_sample(question, image_files):
    args = type('Args', (), {
        "model_path": model_path,
        "model_base": None,
        "model_name": model_name,
        "query": question,
        "conv_mode": None,
        "image_file": image_files,
        "sep": ",",
        "temperature": 0,
        "top_p": None,
        "num_beams": 1,
        "max_new_tokens": 512,
    })()

    return eval(tokenizer, model, image_processor,  args)

def webqa_accuracy(answer, label, Qcate):
    if Qcate == 'color':
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", color_set)
    elif Qcate == 'shape': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", shape_set)
    elif Qcate == 'yesno': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", yesno_set)
    elif Qcate == 'number': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", {"NUMBER"})
    else:
        return None
    return (F1_avg, F1_max, EM, RE_avg, PR_avg)

# def accuracy_agg_results(llava_results):
#     return np.mean([v for k, l in llava_results.items() for v in l])


# def accuracy_agg_results(qa_results):
#     single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
#     two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

#     single_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in single_image_keys])
#     two_image_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in two_image_keys])
#     avr_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items()])
#     return (single_acc, two_image_acc, avr_acc)

# def accuracy_agg_generated_results(qa_results):
#     single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
#     two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

#     single_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in single_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
#     two_image_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in two_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
#     avr_acc = np.mean([PR_avg for key, dict in qa_results.items() for idx, (_,_,_,_,PR_avg) in dict.items()])
    
#     return (single_acc, two_image_acc, avr_acc)

def get_prompt(key, reverse_images = False):
    imgs = eval_data[key]['img_posFacts']
    if len(imgs) == 1:
        return f"<image-placeholder> \n Caption: {imgs[0]['title']} \n Question: {eval_data[key]['Q']}"
    assert(len(imgs) == 2)
    if reverse_images:
        return f"<image-placeholder> \n Caption: {imgs[1]['title']} \n <image-placeholder> \n Caption: {imgs[0]['title']} \n Question: {eval_data[key]['Q']}"
    return f"<image-placeholder> \n Caption: {imgs[0]['title']} \n <image-placeholder> \n Caption: {imgs[1]['title']} \n Question: {eval_data[key]['Q']}"



/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.37s/it]


### LLAVA baseline on original images that have perturbations

In [26]:
from tqdm import tqdm
label= 'yes no'
llava_results_baseline = {}
for k in list(eval_data.keys()):
    question = get_prompt(k)
    # print(question)
    image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
    try:
        answer = llava_eval_on_webqa_sample(question, image_files)
    except:
        continue
        # answer = 'ERROR: llava failed'
    # label = eval_data[k]['A']
    eval_data[k]['A_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_baseline[k] = webqa_accuracy(answer, label, Qcate)
    
    # print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")

print(accuracy_agg_results(llava_results_baseline))

0.0


### Correct despite blank image

In [27]:
llava_results_blank = {}
for k in list(eval_data.keys()):
    question = get_prompt(k)
    image_files ='playground/counterfactual_exp/BLANK.jpg,playground/counterfactual_exp/BLANK.jpg'
    try:
        answer = llava_eval_on_webqa_sample(question, image_files)
    except:
        continue
        # answer = 'ERROR: llava failed'
    label = 'yes no'#eval_data[k]['A']
    eval_data[k]['A_llava_blank'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_blank[k] = webqa_accuracy(answer, label, Qcate)
    # print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")
    
print(accuracy_agg_results(llava_results_blank))

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


0.0


### Correct despite perturbation

In [28]:
llava_results_perturbed_original_label = {}
llava_results_perturbed_any_label = {}
# llava_results_perturbed_generated_label = {}

perturbation_path = "/home/pcarragh/dev/webqa/image_gen_val/val_images_perturbed_gpt_obj_lama"

for k in list(eval_data.keys()):
    eval_data[k]['A_perturbed_llava'] = {}
    question = get_prompt(k)
    image_id = eval_data[k]['img_posFacts'][0]['image_id']
    
    # image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
    image_files = ""
    for img in eval_data[k]['img_posFacts']:
        generated_path = f"{perturbation_path}/{str(image_id)}_{k}.jpeg"
        if os.path.exists(generated_path):
            image_files += "," + generated_path
        else:
            image_files += "," + str(img['image_id'])
    try:
        answer = llava_eval_on_webqa_sample("Answer the following question based on the provided images. \n" + question, image_files[1:])
    except:
        continue
        # answer = 'ERROR: llava failed'

    original_label = eval_data[k]['A']
    eval_data[k]['A_perturbed_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")
    llava_results_perturbed_any_label[k] = webqa_accuracy(answer, 'yes no', Qcate)
    llava_results_perturbed_original_label[k] = webqa_accuracy(answer, original_label, Qcate)

print(accuracy_agg_results(llava_results_perturbed_original_label))
print(accuracy_agg_results(llava_results_perturbed_any_label))

Question: "Does a Minnetonka Rhododendron flower have petals in a cup shape?", Answer: Yes, a Minnetonka Rhododendron flower has petals in a cup shape., Label: yes no
Question: "Are the heads of Iranian women covered in traditional clothing?", Answer: Yes, Label: yes no
Question: "Does only one room in Graceland have light fixtures that hang from the ceiling?", Answer: Yes, Label: yes no
Question: "Does the Osman Shah Mosque have fewer than three support pillars?", Answer: No, Label: yes no
Question: "Was there a building constructed after 2007 that can now be seen in the distance behind the Empire State Building and mimics its shape?", Answer: No, Label: yes no
Question: "Both Expo Milano 2015, Pavilion of China and Spain Pavilion of Shanghai Expo have curved roofs?", Answer: Yes, Label: yes no
Question: "Do the wings of the Bohemian Waxwing project beyond the tip of the tail?", Answer: Yes, Label: yes no
Question: "Does the horse depicted in the Riva Dei Schiavoni Statue have more fe

In [37]:
llava_results_baseline = {}
llava_results_blank = {}
llava_results_perturbed_original_label = {}
llava_results_perturbed_any_label = {}

for k in qids:
    if not k in eval_data or not 'A_perturbed_llava' in eval_data[k] or not 'A_llava' in eval_data[k] or not 'A_llava_blank' in eval_data[k]:
        continue
    perturbed = eval_data[k]['A_perturbed_llava']
    original = eval_data[k]['A_llava']
    blank = eval_data[k]['A_llava_blank']
    def ans_contains_label(ans, labels = ['yes', 'no']):
        return any([label in ans.lower() for label in labels])
    
    llava_results_perturbed_any_label[k] = ans_contains_label(perturbed)
    # llava_results_perturbed_original_label[k] = ans_contains_label(perturbed, ['yes'])
    llava_results_baseline[k] = ans_contains_label(original)
    llava_results_blank[k] = ans_contains_label(blank)    

In [38]:
np.mean(list(llava_results_perturbed_any_label.values())), np.mean(list(llava_results_baseline.values())), np.mean(list(llava_results_blank.values()))

(1.0, 0.7818181818181819, 0.997979797979798, 1.0)

: 

In [35]:
import pandas as pd

baseline_accs = accuracy_agg_results(llava_results_baseline)
blank_accs = accuracy_agg_results(llava_results_blank)
perturbed_original_label_accs = accuracy_agg_results(llava_results_perturbed_original_label)
perturbed_any_label_acc = accuracy_agg_results(llava_results_perturbed_any_label)

columns = ['experiment name', 'single_image', 'two_image', 'average']
accuracy_agg_df = pd.DataFrame(columns=columns)
accuracy_agg_df['experiment name'] = ['baseline', 'blank', 'perturbed_original_label', 'perturbed_any_label']
accuracy_agg_df['average'] = [baseline_accs, blank_accs, perturbed_original_label_accs, perturbed_any_label_acc]

# accuracy_agg_df['single_image'] = [baseline_accs[0], blank_accs[0], perturbed_original_label_accs[0], perturbed_any_label_acc[0]]
# accuracy_agg_df['two_image'] = [baseline_accs[1], blank_accs[1], perturbed_original_label_accs[1], perturbed_any_label_acc[1]]
# accuracy_agg_df['average'] = [baseline_accs[2], blank_accs[2], perturbed_original_label_accs[2], perturbed_any_label_acc[2]]
accuracy_agg_df.to_csv("results/{exp_name}.csv", index=False)
accuracy_agg_df


TypeError: 'bool' object is not iterable